In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
from fastai.text import *   # Quick accesss to NLP functionality
import html

In [16]:
from fastai.text.models.transformer import tfmerXL_lm_config

In [4]:
PATH=Path.home()/'data/wikitext-2-raw'

In [5]:
data = load_data(PATH, bs=8, bptt=256)

In [ ]:
# model = get_language_model(TransformerXL, len(data.vocab.itos))

In [19]:
vocab_sz = len(data.vocab.itos)
config = tfmerXL_lm_config.copy(); config

{'ctx_len': 150,
 'n_layers': 12,
 'n_heads': 10,
 'd_model': 410,
 'd_head': 41,
 'd_inner': 2100,
 'resid_p': 0.1,
 'attn_p': 0.1,
 'ff_p': 0.1,
 'embed_p': 0.1,
 'output_p': 0.1,
 'bias': False,
 'scale': True,
 'act': <Activation.ReLU: 1>,
 'double_drop': True,
 'tie_weights': True,
 'out_bias': True,
 'init': <function fastai.text.models.transformer.init_transformer(m)>,
 'mem_len': 150,
 'mask': True}

In [20]:
init = config.pop('init') if 'init' in config else None
tie_weights,output_p,out_bias = map(config.pop, ['tie_weights', 'output_p', 'out_bias'])
encoder = TransformerXL(vocab_sz, **config)
enc = encoder.encoder if tie_weights else None
decoder = LinearDecoder(vocab_sz, config['d_model'], output_p, tie_encoder=enc, bias=out_bias)
model = SequentialRNN(encoder, decoder)
model.apply(init)

SequentialRNN(
  (0): TransformerXL(
    (encoder): Embedding(39882, 410)
    (pos_enc): PositionalEncoding()
    (drop_emb): Dropout(p=0.1)
    (layers): ModuleList(
      (0): DecoderLayer(
        (mhra): MultiHeadRelativeAttention(
          (attention): Linear(in_features=410, out_features=1230, bias=False)
          (out): Linear(in_features=410, out_features=410, bias=False)
          (drop_att): Dropout(p=0.1)
          (drop_res): Dropout(p=0.1)
          (ln): LayerNorm(torch.Size([410]), eps=1e-05, elementwise_affine=True)
          (r_attn): Linear(in_features=410, out_features=410, bias=False)
        )
        (ff): SequentialEx(
          (layers): ModuleList(
            (0): Linear(in_features=410, out_features=2100, bias=True)
            (1): ReLU(inplace)
            (2): Dropout(p=0.1)
            (3): Linear(in_features=2100, out_features=410, bias=True)
            (4): Dropout(p=0.1)
            (5): MergeLayer()
            (6): LayerNorm(torch.Size([410]), eps

In [21]:
learn = LanguageLearner(data, model)

In [22]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 